In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader

## Load CIFAR-100 and create functions for data split and data loader and tranfromers

In [2]:
# Define data transforms for training and testing
# For training: Random crop and flip for augmentation, then resize to 224 and normalize
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),       # random crop/rescale to 224x224
    transforms.RandomHorizontalFlip(),       # random horizontal flip
    transforms.ToTensor(),                   # convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # normalize with ImageNet mean/std
                         std=[0.229, 0.224, 0.225])
])
# For testing: Resize and center crop to 224, then normalize (no random augmentation)
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

full_trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform =train_transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform = test_transform)


# Split function for 70 60 and 50 percent 

In [3]:

def get_train_val_split(dataset, train_percentage):
    train_len = int(len(dataset) * train_percentage)
    val_len = len(dataset) - train_len
    return random_split(dataset, [train_len, val_len])

trainset_70, valset_70 = get_train_val_split(full_trainset, 0.7)
trainset_60, valset_60 = get_train_val_split(full_trainset, 0.6)
trainset_50, valset_50 = get_train_val_split(full_trainset, 0.5)




# Dataloader

In [5]:
def get_loader(dataset_subset, batch_size=256, shuffle=True):
    return DataLoader(dataset_subset, batch_size=batch_size, shuffle=shuffle, num_workers=2)

trainloader_70 = get_loader(trainset_70, shuffle=True)
valloader_70 = get_loader(valset_70, shuffle=False)

trainloader_60 = get_loader(trainset_60, shuffle=True)
valloader_60 = get_loader(valset_60, shuffle=False)

trainloader_50 = get_loader(trainset_50, shuffle=True)
valloader_50 = get_loader(valset_50, shuffle=False)

testloader = DataLoader(testset, batch_size=256, shuffle=False, num_workers=2)

# Train and evaluation function

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import numpy as np
import copy

def train_one_model(model, trainloader, valloader, testloader, num_epochs=200, lr=0.01, patience=3, min_delta=0.001,optimizer=None):
    #check device
    if torch.cuda.is_available():
        device = 'cuda'
        print("Using GPU for training.")
    else:
        device = 'cpu'
        print("Using CPU for training.")
        
    device = torch.device(device)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    if optimizer is None:
        print("Optimizer not provided, using SGD with default parameters.")
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    if optimizer == 'adam':
        print("Using Adam optimizer.")
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    best_loss = float('inf')
    best_model_state = None
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        print(f'\n Epoch {epoch+1}/{num_epochs} | LR: {scheduler.get_last_lr()[0]:.6f}')
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, targets in tqdm(trainloader, desc="Training", leave=False):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        acc = 100. * correct / total
        avg_train_loss = running_loss / len(trainloader)
        print(f' Train Accuracy: {acc:.2f}% | Loss: {avg_train_loss:.4f}')
        scheduler.step()

        # ===== Validation για early stopping =====
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for inputs, targets in valloader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += targets.size(0)
                val_correct += predicted.eq(targets).sum().item()

        avg_val_loss = val_loss / len(valloader)
        val_acc = 100. * val_correct / val_total
        print(f'Validation Accuracy: {val_acc:.2f}% | Loss: {avg_val_loss:.4f}')

        if best_loss - avg_val_loss > min_delta:
            best_loss = avg_val_loss
            best_model_state = copy.deepcopy(model.state_dict())
            epochs_no_improve = 0
            print(" New best model saved.")
        else:
            epochs_no_improve += 1
            print(f" No significant improvement (> {min_delta}) for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print(f" Early stopping triggered after {epoch+1} epochs.")
            break

    # ===== Φόρτωσε το καλύτερο μοντέλο =====
    if best_model_state:
        model.load_state_dict(best_model_state)

    # ===== Τελική αξιολόγηση στο test set =====
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    accuracy = accuracy_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds, average='macro')
    recall = recall_score(all_targets, all_preds, average='macro')
    f1 = f1_score(all_targets, all_preds, average='macro')

    print("\n Classification Metrics (Best Model on Test Set):")
    print(f" Accuracy:      {accuracy:.4f}")
    print(f" Precision:     {precision:.4f}")
    print(f" Recall:        {recall:.4f}")
    print(f" F1 Score:      {f1:.4f}")

    return model, {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }


# Create models

## 1st model ResNet

In [7]:
import torchvision.models as models
import torch.nn as nn

def create_resnet18(num_classes=100, pretrained=True):
    model = models.resnet18(pretrained=pretrained)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

def create_shufflenet_v2(num_classes=100, pretrained=True):
    model = models.shufflenet_v2_x1_0(pretrained=pretrained)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model


In [ ]:
# train ResNet-18 με 70% εκπαίδευση
print("Training ResNet-18 with 70% training data...")
resnet18 = create_resnet18(num_classes=100, pretrained=True)
resnet18_70, resnet18_metrics_70 = train_one_model(
    resnet18, trainloader_70, valloader_70, testloader, num_epochs=30, lr=0.01, patience=3, min_delta=0.001
)

Training ResNet-18 with 70% training data...
Using GPU for training.


c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



 Epoch 1/30 | LR: 0.010000


 Train Accuracy: 25.70% | Loss: 3.1405
Validation Accuracy: 42.75% | Loss: 2.2119
 New best model saved.

 Epoch 2/30 | LR: 0.009973


 Train Accuracy: 49.82% | Loss: 1.9032
Validation Accuracy: 51.13% | Loss: 1.8232
 New best model saved.

 Epoch 3/30 | LR: 0.009891


...

 Train Accuracy: 81.23% | Loss: 0.7063
Validation Accuracy: 67.04% | Loss: 1.2067
 New best model saved.

 Epoch 25/30 | LR: 0.000955


 Train Accuracy: 81.15% | Loss: 0.7108
Validation Accuracy: 67.83% | Loss: 1.1777
 New best model saved.

 Epoch 26/30 | LR: 0.000670


 Train Accuracy: 82.07% | Loss: 0.6843
Validation Accuracy: 67.44% | Loss: 1.1868
 No significant improvement (> 0.001) for 1 epoch(s).

 Epoch 27/30 | LR: 0.000432


 Train Accuracy: 81.83% | Loss: 0.6972
Validation Accuracy: 67.18% | Loss: 1.1910
 No significant improvement (> 0.001) for 2 epoch(s).

 Epoch 28/30 | LR: 0.000245


 Train Accuracy: 81.81% | Loss: 0.6862
Validation Accuracy: 67.35% | Loss: 1.1864
 No significant improvement (> 0.001) for 3 epoch(s).
 Early stopping triggered after 28 epochs.

 Classification Metrics (Best Model on Test Set):
 Accuracy:      0.8024
 Precision:     0.8044
 Recall:        0.8024
 F1 Score:      0.8022


In [8]:
# train ResNet-18 με 60% εκπαίδευση
print("Training ResNet-18 with 60% training data...")
resnet18 = create_resnet18(num_classes=100, pretrained=True)
resnet18_60, resnet18_metrics_60 = train_one_model(
    resnet18, trainloader_60, valloader_60, testloader, num_epochs=30, lr=0.01,  patience=3, min_delta=0.001
)

c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training ResNet-18 with 60% training data...
Using GPU for training.

 Epoch 1/30 | LR: 0.010000


 Train Accuracy: 24.37% | Loss: 3.2470
Validation Accuracy: 42.02% | Loss: 2.2685
 New best model saved.

 Epoch 2/30 | LR: 0.009973


 Train Accuracy: 48.38% | Loss: 1.9665
Validation Accuracy: 51.03% | Loss: 1.8417
 New best model saved.

 Epoch 3/30 | LR: 0.009891


 Train Accuracy: 55.74% | Loss: 1.6631
Validation Accuracy: 53.66% | Loss: 1.7194
 New best model saved.

 Epoch 4/30 | LR: 0.009755


...

 Train Accuracy: 81.35% | Loss: 0.7112
Validation Accuracy: 66.61% | Loss: 1.2062
 No significant improvement (> 0.001) for 1 epoch(s).

 Epoch 27/30 | LR: 0.000432


 Train Accuracy: 81.96% | Loss: 0.6868
Validation Accuracy: 66.89% | Loss: 1.1991
 New best model saved.

 Epoch 28/30 | LR: 0.000245


 Train Accuracy: 82.02% | Loss: 0.6925
Validation Accuracy: 66.94% | Loss: 1.2118
 No significant improvement (> 0.001) for 1 epoch(s).

 Epoch 29/30 | LR: 0.000109


 Train Accuracy: 81.80% | Loss: 0.6996
Validation Accuracy: 66.67% | Loss: 1.2134
 No significant improvement (> 0.001) for 2 epoch(s).

 Epoch 30/30 | LR: 0.000027


 Train Accuracy: 82.20% | Loss: 0.6766
Validation Accuracy: 66.57% | Loss: 1.2168
 No significant improvement (> 0.001) for 3 epoch(s).
 Early stopping triggered after 30 epochs.

 Classification Metrics (Best Model on Test Set):
 Accuracy:      0.7981
 Precision:     0.8013
 Recall:        0.7981
 F1 Score:      0.7985


In [8]:
# train ResNet-18 με 50% εκπαίδευση
print("Training ResNet-18 with 50% training data...")
resnet18 = create_resnet18(num_classes=100, pretrained=True)
resnet18_50, resnet18_metrics_50 = train_one_model(
    resnet18, trainloader_50, valloader_50, testloader, num_epochs=30, lr=0.01, patience=3, min_delta=0.001 
)

c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training ResNet-18 with 50% training data...
Using GPU for training.
Optimizer not provided, using SGD with default parameters.

 Epoch 1/30 | LR: 0.010000


 Train Accuracy: 19.99% | Loss: 3.5051
Validation Accuracy: 35.24% | Loss: 2.5544
 New best model saved.

 Epoch 2/30 | LR: 0.009973


 Train Accuracy: 45.06% | Loss: 2.1357
Validation Accuracy: 47.34% | Loss: 2.0090
 New best model saved.

 Epoch 3/30 | LR: 0.009891


 Train Accuracy: 52.88% | Loss: 1.7719
Validation Accuracy: 52.12% | Loss: 1.7856
 New best model saved.

 Epoch 4/30 | LR: 0.009755


...

 Train Accuracy: 81.29% | Loss: 0.7199
Validation Accuracy: 64.76% | Loss: 1.2876
 No significant improvement (> 0.001) for 1 epoch(s).

 Epoch 27/30 | LR: 0.000432


 Train Accuracy: 81.17% | Loss: 0.7125
Validation Accuracy: 64.91% | Loss: 1.2757
 New best model saved.

 Epoch 28/30 | LR: 0.000245


 Train Accuracy: 81.40% | Loss: 0.7143
Validation Accuracy: 65.20% | Loss: 1.2826
 No significant improvement (> 0.001) for 1 epoch(s).

 Epoch 29/30 | LR: 0.000109


 Train Accuracy: 81.43% | Loss: 0.7085
Validation Accuracy: 64.97% | Loss: 1.2762
 No significant improvement (> 0.001) for 2 epoch(s).

 Epoch 30/30 | LR: 0.000027


 Train Accuracy: 81.61% | Loss: 0.7069
Validation Accuracy: 65.56% | Loss: 1.2662
 New best model saved.

 Classification Metrics (Best Model on Test Set):
 Accuracy:      0.7821
 Precision:     0.7849
 Recall:        0.7821
 F1 Score:      0.7824


In [11]:
# train ShuffleNet V2 με 70% εκπαίδευση
print("Training ShuffleNet V2 with 70% training data...")   
shufflenet_v2 = create_shufflenet_v2(num_classes=100, pretrained=True)
shufflenet_v2_70, shufflenet_v2_metrics_70 = train_one_model(
    shufflenet_v2, trainloader_70, valloader_70, testloader, num_epochs=100, lr=0.01, patience=10, min_delta=0.001
)

c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training ShuffleNet V2 with 70% training data...
Using GPU for training.
Optimizer not provided, using SGD with default parameters.

 Epoch 1/100 | LR: 0.010000


 Train Accuracy: 1.83% | Loss: 4.6027
Validation Accuracy: 2.42% | Loss: 4.6002
 New best model saved.

 Epoch 2/100 | LR: 0.009998


 Train Accuracy: 3.54% | Loss: 4.5965
Validation Accuracy: 4.32% | Loss: 4.5939
 New best model saved.

 Epoch 3/100 | LR: 0.009990


 Train Accuracy: 5.84% | Loss: 4.5884
Validation Accuracy: 5.65% | Loss: 4.5845
 New best model saved.

 Epoch 4/100 | LR: 0.009978


...

 Train Accuracy: 77.23% | Loss: 0.8438
Validation Accuracy: 62.51% | Loss: 1.3706
 No significant improvement (> 0.001) for 6 epoch(s).

 Epoch 86/100 | LR: 0.000545


 Train Accuracy: 77.83% | Loss: 0.8323
Validation Accuracy: 63.20% | Loss: 1.3473
 No significant improvement (> 0.001) for 7 epoch(s).

 Epoch 87/100 | LR: 0.000476


 Train Accuracy: 77.58% | Loss: 0.8411
Validation Accuracy: 63.00% | Loss: 1.3488
 No significant improvement (> 0.001) for 8 epoch(s).

 Epoch 88/100 | LR: 0.000411


 Train Accuracy: 78.05% | Loss: 0.8250
Validation Accuracy: 63.13% | Loss: 1.3449
 No significant improvement (> 0.001) for 9 epoch(s).

 Epoch 89/100 | LR: 0.000351


 Train Accuracy: 77.55% | Loss: 0.8352
Validation Accuracy: 63.05% | Loss: 1.3448
 No significant improvement (> 0.001) for 10 epoch(s).
 Early stopping triggered after 89 epochs.

 Classification Metrics (Best Model on Test Set):
 Accuracy:      0.7576
 Precision:     0.7597
 Recall:        0.7576
 F1 Score:      0.7574


In [9]:
# train ShuffleNet V2 με 60% εκπαίδευση
print("Training ShuffleNet V2 with 60% training data...")
shufflenet_v2 = create_shufflenet_v2(num_classes=100, pretrained=True)
shufflenet_v2_60, shufflenet_v2_metrics_60 = train_one_model(
    shufflenet_v2, trainloader_60, valloader_60, testloader, num_epochs=100, lr=0.01, patience=10, min_delta=0.001
)

c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training ShuffleNet V2 with 60% training data...
Using GPU for training.
Optimizer not provided, using SGD with default parameters.

 Epoch 1/100 | LR: 0.010000


 Train Accuracy: 1.15% | Loss: 4.6038
Validation Accuracy: 1.53% | Loss: 4.6015
 New best model saved.

 Epoch 2/100 | LR: 0.009998


 Train Accuracy: 2.38% | Loss: 4.5985
Validation Accuracy: 2.98% | Loss: 4.5963
 New best model saved.

 Epoch 3/100 | LR: 0.009990


 Train Accuracy: 3.87% | Loss: 4.5922
Validation Accuracy: 4.54% | Loss: 4.5898
 New best model saved.

 Epoch 4/100 | LR: 0.009978


...

 Train Accuracy: 75.87% | Loss: 0.8967
Validation Accuracy: 60.87% | Loss: 1.4251
 No significant improvement (> 0.001) for 4 epoch(s).

 Epoch 97/100 | LR: 0.000039


 Train Accuracy: 76.19% | Loss: 0.8829
Validation Accuracy: 61.16% | Loss: 1.4165
 No significant improvement (> 0.001) for 5 epoch(s).

 Epoch 98/100 | LR: 0.000022


 Train Accuracy: 76.15% | Loss: 0.8877
Validation Accuracy: 61.27% | Loss: 1.4260
 No significant improvement (> 0.001) for 6 epoch(s).

 Epoch 99/100 | LR: 0.000010


 Train Accuracy: 76.11% | Loss: 0.8942
Validation Accuracy: 61.00% | Loss: 1.4257
 No significant improvement (> 0.001) for 7 epoch(s).

 Epoch 100/100 | LR: 0.000002


 Train Accuracy: 76.34% | Loss: 0.8863
Validation Accuracy: 61.51% | Loss: 1.4148
 No significant improvement (> 0.001) for 8 epoch(s).

 Classification Metrics (Best Model on Test Set):
 Accuracy:      0.7417
 Precision:     0.7442
 Recall:        0.7417
 F1 Score:      0.7418


In [12]:
# train ShuffleNet V2 με 50% εκπαίδευση
print("Training ShuffleNet V2 with 50% training data...")
shufflenet_v2 = create_shufflenet_v2(num_classes=100, pretrained=True)
shufflenet_v2_50, shufflenet_v2_metrics_50 = train_one_model(
    shufflenet_v2, trainloader_50, valloader_50, testloader, num_epochs=100, lr=0.01, patience=10, min_delta=0.001
)

c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mateo\anaconda3\envs\cifar100\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training ShuffleNet V2 with 50% training data...
Using GPU for training.
Optimizer not provided, using SGD with default parameters.

 Epoch 1/100 | LR: 0.010000


 Train Accuracy: 1.10% | Loss: 4.6039
Validation Accuracy: 1.42% | Loss: 4.6022
 New best model saved.

 Epoch 2/100 | LR: 0.009998


 Train Accuracy: 1.44% | Loss: 4.5996
Validation Accuracy: 1.62% | Loss: 4.5980
 New best model saved.

 Epoch 3/100 | LR: 0.009990


 Train Accuracy: 2.70% | Loss: 4.5950
Validation Accuracy: 2.94% | Loss: 4.5932
 New best model saved.

 Epoch 4/100 | LR: 0.009978


...

 Train Accuracy: 73.02% | Loss: 0.9984
Validation Accuracy: 59.38% | Loss: 1.4745
 No significant improvement (> 0.001) for 5 epoch(s).

 Epoch 97/100 | LR: 0.000039


 Train Accuracy: 73.53% | Loss: 0.9947
Validation Accuracy: 59.74% | Loss: 1.4761
 No significant improvement (> 0.001) for 6 epoch(s).

 Epoch 98/100 | LR: 0.000022


 Train Accuracy: 73.43% | Loss: 0.9926
Validation Accuracy: 59.40% | Loss: 1.4796
 No significant improvement (> 0.001) for 7 epoch(s).

 Epoch 99/100 | LR: 0.000010


 Train Accuracy: 73.16% | Loss: 0.9963
Validation Accuracy: 59.70% | Loss: 1.4775
 No significant improvement (> 0.001) for 8 epoch(s).

 Epoch 100/100 | LR: 0.000002


 Train Accuracy: 73.45% | Loss: 0.9899
Validation Accuracy: 59.72% | Loss: 1.4838
 No significant improvement (> 0.001) for 9 epoch(s).

 Classification Metrics (Best Model on Test Set):
 Accuracy:      0.7212
 Precision:     0.7233
 Recall:        0.7212
 F1 Score:      0.7206


In [9]:
print ("model architecture ResNet-18 with 70% training data")
print(resnet18_50)

print ("model architecture shufflenet_v2 with 70% training data")
print(shufflenet_v2_70)

model architecture ResNet-18 with 70% training data
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

NameError: name 'shufflenet_v2_70' is not defined